In [ ]:
!pip install --quiet transformers==4.5.0
!pip install --quiet pytorch-lightning==1.2.7
!pip install --quiet sentencepiece
import torch
from transformers import MT5ForConditionalGeneration, T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/mt5-large")
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-large')

     |████████████████████████████████| 2.1 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 31.7 MB/s 
     |████████████████████████████████| 895 kB 47.5 MB/s 
     |████████████████████████████████| 830 kB 5.3 MB/s 
     |████████████████████████████████| 125 kB 43.6 MB/s 
     |████████████████████████████████| 596 kB 21.9 MB/s 
     |████████████████████████████████| 282 kB 45.2 MB/s 
     |████████████████████████████████| 829 kB 31.9 MB/s 
     |████████████████████████████████| 1.3 MB 25.9 MB/s 
     |████████████████████████████████| 160 kB 46.5 MB/s 
     |████████████████████████████████| 271 kB 47.6 MB/s 
     |████████████████████████████████| 1.2 MB 5.2 MB/s 


Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

# Copia el archivo desde el drive al entorno de ejecucion

shutil.copy("./drive/MyDrive/spa_news_2020_1M.tar.gz", "./spa_news_2020_1M.tar.gz")

'./spa_news_2020_1M.tar.gz'

In [ ]:
# importing the "tarfile" module
import tarfile
  
# open file
file = tarfile.open('spa_news_2020_1M.tar.gz')
  
# extracting file
file.extractall('.')
  
file.close()

In [ ]:
def msize(m):
    return sum(p.numel() for p in m.parameters())
print(msize(model.shared) / msize(model))   
print(msize(model.lm_head) / msize(model))  

0.2082942262544732
0.2082942262544732


In [ ]:
import pandas as pd
import csv
from collections import Counter
from tqdm.auto import tqdm, trange
df_es = pd.read_csv('./spa_news_2020_1M/spa_news_2020_1M-sentences.txt', sep='\t', header=None, quoting=csv.QUOTE_NONE)
df_es.columns = ['idx', 'text']
cnt_es = Counter()
for text in tqdm(df_es.text):
    cnt_es.update(tokenizer.encode(text))
print(len(cnt_es), len(cnt_es)/tokenizer.vocab_size)  


  0%|          | 0/1000000 [00:00<?, ?it/s]

64840 0.2592562974810076


In [ ]:
for top in 10_000, 20_000, 30_000:
    print(top, sum(v for k, v in cnt_es.most_common(top)) / sum(cnt_es.values()))


10000 0.9709232293434434
20000 0.9908633305930422
30000 0.996493022559441


In [ ]:
cnt_es.most_common(2)

[(259, 4129329), (269, 1622306)]

In [ ]:
tokenizer.decode(269)

'de'

In [ ]:
#with open(f'masComunesESP.txt', 'w', encoding='utf-8') as fp:
#    fp.write(''.join(cnt_es.most_common(25_000)))

In [ ]:
new_tokens = set(range(1000))
#for i, (k, v) in enumerate(cnt_en.most_common(10_000)):
#    if k not in new_tokens:
#        new_tokens.add(k)
for i, (k, v) in enumerate(cnt_es.most_common(25_000)):
    if len(new_tokens) == 29_900:
        print(i, 'Esp tokens are included')
        break
    if k not in new_tokens:
        new_tokens.add(k)
for t in range(tokenizer.vocab_size - 100, tokenizer.vocab_size):
    new_tokens.add(t)
print(len(new_tokens))
kept_ids = sorted(new_tokens)

25620


In [ ]:
new_size = len(kept_ids)
new_emb = torch.nn.Embedding(new_size, model.shared.embedding_dim)
new_head = torch.nn.Linear(in_features=model.lm_head.in_features, out_features=new_size, bias=False)
for new_id, old_id in enumerate(kept_ids):
    new_emb.weight.data[new_id] = model.shared.weight.data[old_id]
    new_head.weight.data[new_id] = model.lm_head.weight.data[old_id]
model.shared.weight = new_emb.weight
model.lm_head.weight = new_head.weight
model.config.__dict__['vocab_size'] = new_size
model.config.__dict__['_name_or_path'] = 'cointegrated/es5-large'

In [ ]:
! wget https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto
! protoc --python_out=. sentencepiece_model.proto
import sentencepiece_model_pb2 as spmp
smp = tokenizer.sp_model.serialized_model_proto()
m = spmp.ModelProto()
m.ParseFromString(smp)
print('the loaded model has pieces:', len(m.pieces))
new_pieces = [m.pieces[idx] for idx in kept_ids]
print('the new pieces:', len(new_pieces))
# replace the content of the first 30K pieces
for i, p in enumerate(new_pieces):
    m.pieces[i].piece = p.piece
    m.pieces[i].score = p.score
    m.pieces[i].type = p.type
# drop the remaining pieces
n = len(new_pieces)
for i in trange(len(m.pieces) - n):
    m.pieces.pop(len(m.pieces) - 1)
print(len(m.pieces))
with open('new_sp.model', 'wb') as f:
    f.write(m.SerializeToString())
new_tokenizer = T5Tokenizer('new_sp.model', extra_ids=100)

--2021-10-15 14:13:55--  https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12872 (13K) [text/plain]
Saving to: ‘sentencepiece_model.proto’

sentencepiece_model 100%[===================>]  12.57K  --.-KB/s    in 0s      

2021-10-15 14:13:55 (89.3 MB/s) - ‘sentencepiece_model.proto’ saved [12872/12872]

the loaded model has pieces: 250100
the new pieces: 25620


  0%|          | 0/224480 [00:00<?, ?it/s]

25620


In [ ]:
new_tokenizer.save_pretrained('espt5-large')
model.save_pretrained('espt5-large')

In [ ]:
shutil.copytree("./espt5-large", "./drive/MyDrive/espt5-large")

'./drive/MyDrive/espt5-large'